# 2023_04_17

지금은 geohash의 `precision = 7`로 해서 데이터를 생성함  
도시를 소도시로 잡아서 그런가 weather data와 accient data를 병합이 되는 데이터가 적음  
`precision = 5` or `precision = 6`에 대해서 모든 데이터를 생성하고 데이터가 얼마나 생성되는지 확인  
이때 weather data가 1시간 단위가 아니라 구간으로 되어 있어서 1시간 단위로 분리하는 작업이 따로 필요함  


> validation code : http://202.31.200.70:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/Preproecssing_data_all_city_validation.ipynb

다음과 같이 python 코드로 해서 실행
360시간 정도 시간이 소요됨  

```python
# 필요한 패키지 로드
import geohash
import folium
import geohash
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm , tqdm_notebook
import pandas as pd
from shapely.geometry import Polygon
pd.set_option('mode.chained_assignment',  None)
import warnings
warnings.filterwarnings('ignore', message='The frame.append method is deprecated')

# 데이터 불러오기
df = pd.read_csv('./data/TrafficEvents_Aug16_Dec20_Publish.csv')

weather_df = pd.read_csv('./data/WeatherEvents_Aug16_Dec20_Publish.csv')

union_city = list(set(df['City'].unique()) & set(weather_df['City'].unique()))

precision = 5
city_merge_dfs = []

for city in tqdm(union_city[1:]):
    # Accident data
    city_df = df[df['City'] == city]
    city_df['StartTime(UTC)'] = pd.to_datetime(city_df['StartTime(UTC)'])
    city_df['EndTime(UTC)'] = pd.to_datetime(city_df['EndTime(UTC)'])
    city_df['OccTime'] = city_df['StartTime(UTC)'].apply(lambda x: x.replace(minute=0, second=0))
    city_df['EndTime'] = city_df['EndTime(UTC)'].apply(lambda x: x.replace(minute=0, second=0))
    city_df['GeoHash'] = city_df.apply(lambda row: geohash.encode(row['LocationLat'], row['LocationLng'], precision=precision), axis=1)
    
    insert_columns = ['OccTime', 'EndTime', 'GeoHash', 'Type', 'Severity', 'EventId', 'Distance(mi)', 'Description']
    count_df = pd.DataFrame(columns=insert_columns)
    
    for e_id in tqdm(city_df['EventId'], desc = f'{city} Traffic'):
        event_df = city_df[city_df['EventId'] == e_id][insert_columns]
        dt = pd.date_range(event_df['OccTime'].iloc[0], event_df['EndTime'].iloc[0], freq='1h')
        for d in dt:
            count_df = count_df.append(event_df.iloc[0], ignore_index=True)

    count_df = count_df[count_df['OccTime'].notnull()]
    
    # Weather data
    weather_city_df = weather_df[weather_df['City'] == city]
    weather_city_df['StartTime(UTC)'] = pd.to_datetime(weather_city_df['StartTime(UTC)'])
    weather_city_df['EndTime(UTC)'] = pd.to_datetime(weather_city_df['EndTime(UTC)'])
    weather_city_df['OccTime'] = weather_city_df['StartTime(UTC)'].apply(lambda x: x.replace(minute=0, second=0))
    weather_city_df['EndTime'] = weather_city_df['EndTime(UTC)'].apply(lambda x: x.replace(minute=0, second=0))
    weather_city_df['GeoHash'] = weather_city_df.apply(lambda row: geohash.encode(row['LocationLat'], row['LocationLng'], precision=precision), axis=1)
    
    weather_insert_columns = ['OccTime', 'EndTime', 'Type', 'GeoHash']
    weather_count_df = pd.DataFrame(columns=weather_insert_columns)
    
    for e_id in tqdm(weather_city_df['EventId'],desc =f'{city} weather'):
        weather_event_df = weather_city_df[weather_city_df['EventId'] == e_id][weather_insert_columns]
        dt = pd.date_range(weather_event_df['OccTime'].iloc[0], weather_event_df['EndTime'].iloc[0], freq='1h')
        for d in dt:
            weather_count_df = weather_count_df.append(weather_event_df.iloc[0], ignore_index=True)

    weather_count_df = weather_count_df[weather_count_df['OccTime'].notnull()]
    
    # Merge accident and weather data
    result_df = pd.merge(count_df, weather_count_df, how='inner', on=['OccTime', 'GeoHash'])
    if len(result_df) > 1:
        result_df.to_csv(f'./result_csv/{precision}/{city}_{precision}.csv' ,index=False)

```

주피터 노트북에서는 15시간 정도가 소요되는걸로 나왔으나 python으로 실행시키니 360 시간정도가 소요됨    
이는 현실적으로 하기가 불가능함  

인구 데이터와 공항을 기반으로 적절한 도시 선택  필요  